In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak
from sklearn.model_selection import train_test_split
import jinja2
from pycaret.classification import *

ModuleNotFoundError: No module named 'pycaret'

In [2]:
df = pd.read_csv('../dataset/master_3.csv')
# df.head()

In [12]:
# drop_df = df.drop(['outcome_critical'], axis=1)
# x_train, x_test, y_train, y_test = train_test_split(drop_df, df['outcome_critical'], test_size=0.5, shuffle=True, random_state=2022)

In [8]:
drop_df = df.drop(list(df.filter(regex='cci_').columns), axis=1)

In [13]:
corr = df.corr(method='pearson')['outcome_critical']
corr.sort_values(ascending=False)

outcome_critical                1.000000
outcome_icu_transfer_12h        0.971288
outcome_inhospital_mortality    0.385000
outcome_hospitalization         0.277768
n_med                           0.202126
ed_resprate_last                0.200978
ed_heartrate_last               0.157348
age                             0.153257
triage_resprate                 0.131355
n_icu_365d                      0.124064
n_icu_90d                       0.113711
cci_CHF                         0.085129
eci_CHF                         0.085114
n_icu_30d                       0.084543
eci_FluidsLytes                 0.076142
triage_heartrate                0.075372
eci_Arrhythmia                  0.073491
eci_HTN1                        0.068138
eci_Coagulopathy                0.064700
cci_Renal                       0.064045
eci_Renal                       0.063832
ed_temperature_last             0.060720
eci_PHTN                        0.056837
cci_PVD                         0.053254
eci_PVD         

In [14]:
drop_df = drop_df.drop(['outcome_icu_transfer_12h'], axis=1)

In [17]:
drop_df = drop_df.drop(['chiefcom_chest_pain', 'chiefcom_abdominal_pain',
    'chiefcom_headache', 'chiefcom_shortness_of_breath',
    'chiefcom_back_pain', 'chiefcom_cough', 'chiefcom_nausea_vomiting',
    'chiefcom_fever_chills', 'chiefcom_syncope', 'chiefcom_dizziness',
    'eci_CHF', 'eci_Arrhythmia', 'eci_Valvular', 'eci_PHTN', 'eci_PVD',
    'eci_HTN1', 'eci_HTN2', 'eci_Paralysis', 'eci_NeuroOther',
    'eci_Pulmonary', 'eci_DM1', 'eci_DM2', 'eci_Hypothyroid', 'eci_Renal',
    'eci_Liver', 'eci_PUD', 'eci_HIV', 'eci_Lymphoma', 'eci_Tumor2',
    'eci_Tumor1', 'eci_Rheumatic', 'eci_Coagulopathy', 'eci_Obesity',
    'eci_WeightLoss', 'eci_FluidsLytes', 'eci_BloodLoss', 'eci_Anemia',
    'eci_Alcohol', 'eci_Drugs', 'eci_Psychoses', 'eci_Depression'], axis=1)

In [18]:
drop_df.columns

Index(['index', 'subject_id', 'stay_id', 'intime', 'outtime', 'gender',
       'in_year', 'age', 'outcome_inhospital_mortality', 'ed_death',
       'before_ed_mortality', 'ed_los', 'outcome_hospitalization',
       'outcome_critical', 'n_ed_30d', 'n_ed_90d', 'n_ed_365d',
       'outcome_ed_revisit_3d', 'n_hosp_30d', 'n_hosp_90d', 'n_hosp_365d',
       'n_icu_30d', 'n_icu_90d', 'n_icu_365d', 'ed_los_hours',
       'triage_temperature', 'triage_heartrate', 'triage_resprate',
       'triage_o2sat', 'triage_sbp', 'triage_dbp', 'triage_pain',
       'triage_acuity', 'ed_temperature_last', 'ed_heartrate_last',
       'ed_resprate_last', 'ed_o2sat_last', 'ed_sbp_last', 'ed_dbp_last',
       'ed_pain_last', 'n_med', 'n_medrecon'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

import numpy as np

# 스태킹 모델에 사용할 알고리즘
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# 위스콘신 유방암 예제 데이터 로드
# metrics로 accuracy를 사용
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
obj = pd.DataFrame(drop_df.dtypes != 'object')
drop_df = drop_df[obj[obj[0] == True].index]

In [ ]:
y = drop_df['outcome_critical']
x = drop_df.drop(['outcome_critical'], axis=1)

In [ ]:
#언더 샘플링
undersample = RandomUnderSampler(sampling_strategy='majority')
x_under, y_under = undersample.fit_resample(x, y)
print(Counter(y_under))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_under, y_under, shuffle=True, stratify=y_under, test_size=0.2, random_state=10)

## 일반 스태킹

In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=30)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)
lr_final = LogisticRegression(C=10)

In [ ]:
knn_clf.fit(x_train, y_train)
rf_clf.fit(x_train, y_train)
dt_clf.fit(x_train, y_train)
ada_clf.fit(x_train, y_train)

In [ ]:
knn_pred = knn_clf.predict(x_test)
rf_pred = rf_clf.predict(x_test)
dt_pred = dt_clf.predict(x_test)
ada_pred = ada_clf.predict(x_test)

print('KNN 정확도 :',accuracy_score(y_test, knn_pred))
print('RF 정확도 :',accuracy_score(y_test, rf_pred))
print('DT 정확도 :',accuracy_score(y_test, dt_pred))
print('ADA부스트 정확도 :',accuracy_score(y_test, ada_pred))

In [ ]:
stacked_pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(stacked_pred.shape)

# transpose를 이용, 행과 열의 위치를 교환, 칼럼 레벨로 각 모델의 예측 결과를 피처로 사용

stacked_pred = np.transpose(stacked_pred)

In [ ]:
lr_final.fit(stacked_pred, y_test)
final_pred = lr_final.predict(stacked_pred)

print('최종 메타 모델 정확도 : ',accuracy_score(y_test, final_pred))

## 스태킹 with CV

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    # 지정된 n_folds 값으로 KFold 생성
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=20)
    
    # 추후 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__,' model 시작')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        print('\t 폴드 세트: ',folder_counter+1,' 시작')
        X_tr = X_train_n.iloc[train_index]
        y_tr = y_train_n.iloc[train_index]
        X_te = X_train_n.iloc[valid_index]
        
        # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
        model.fit(X_tr, y_tr)
        # 폴드 세트 내부에서 다시 만들어지 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        # 입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장
        test_pred[:, folder_counter] = model.predict(X_test_n)
        
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    # train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred, test_pred_mean

In [ ]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, x_train, y_train, x_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, x_train, y_train, x_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, x_train, y_train, x_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, x_train, y_train, x_test, 7)

In [ ]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print('원본 학습 피처 데이터 shape:', x_train.shape, '원본 테스트 피처 shape:',x_test.shape)
print('스태킹 학습 피처 데이터 shape:',Stack_final_X_train.shape,
     '스태킹 테스트 피처 데이터 shape:',Stack_final_X_test.shape)

In [ ]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도: ', accuracy_score(y_test, stack_final))